# Data Augmentation: Geometric Transformations


## Setup


In [1]:
import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import (
    Sequential,
    layers,
    losses,
    optimizers,
    regularizers,
    callbacks,
)
import tensorflow_hub as hub

In [2]:
version_ = tf.version.VERSION
print(f'Tensorflow version: {version_}')

Tensorflow version: 2.9.1


In [3]:
device_name = tf.test.gpu_device_name()

if device_name == "":
    BATCH_SIZE = 32
    raise SystemError("No GPU found!")
else:
    BATCH_SIZE = 512
    print(f"GPU found -> {device_name}")

os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"

GPU found -> /device:GPU:0
Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



In [4]:
functions_path = os.path.join("..", "..", "..", "functions")
sys.path.append(functions_path)
import learning_rate_functions


## Constants

In [5]:
IMG_HEIGHT = 448
IMG_WIDTH = 448
IMG_CHANNELS = 3
CLASS_NAMES = 'daisy dandelion rose sunflower tulip'.split()

MODEL_URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
TRAIN_URL = "gs://practical-ml-vision-book/flowers_tfr/train-*"
VALID_URL = "gs://practical-ml-vision-book/flowers_tfr/valid-*"
IMAGES_LIST = [
    "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg",
    "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg",
    "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/98992760_53ed1d26a9.jpg",
    "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9939430464_5f5861ebab.jpg",
    "gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9965757055_ff01b5ee6f_n.jpg",
]

AUTOTUNE = tf.data.AUTOTUNE

## Helper Functions

In [6]:
def get_logdir():
    run_id = time.strftime('run_%Y%m%d-%H%M%S')
    log_dir = os.path.join('..', '..', 'reports', 'logs', run_id)
    return log_dir

In [7]:
class _Preprocessor:
    def __init__(self):
        pass

    def read_from_tfr(self, proto):
        features_description = {
            "image": tf.io.VarLenFeature(tf.float32),
            "shape": tf.io.VarLenFeature(tf.int64),
            "label": tf.io.FixedLenFeature([], tf.string, default_value=""),
            "label_int": tf.io.FixedLenFeature([], tf.int64, default_value=0),
        }
        record = tf.io.parse_single_example(proto, features_description)
        shape = tf.sparse.to_dense(record["shape"])
        image = tf.reshape(tf.sparse.to_dense(record["image"]), shape)
        label_int = record["label_int"]
        return image, label_int

    def read_from_jpeg(self, filename):
        image = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image, channels=IMG_CHANNELS)
        image = tf.image.convert_image_dtype(image, tf.float32)
        return image

    def preprocess(self, image):
        image = tf.image.resize_with_pad(image, IMG_HEIGHT, IMG_WIDTH)
        return image


In [8]:
def preprocessed_dataset(pattern):
    preprocessor = _Preprocessor()
    dataset = (
        tf.data.TFRecordDataset(
            [file for file in tf.io.gfile.glob(pattern)], compression_type="GZIP"
        )
        .map(preprocessor.read_from_tfr, num_parallel_calls=AUTOTUNE)
        .map(lambda image, label: (preprocessor.preprocess(image), label), num_parallel_calls=AUTOTUNE)
    )
    return dataset


def preprocessed_image(filename):
    preprocessor = _Preprocessor()
    image = preprocessor.read_from_jpeg(filename)
    image = preprocessor.preprocess(image)
    return image    


## Transfer Learning with MobileNet

With random cropping and left-right flipping:

In [9]:
NUM_EPOCHS = 150


def train_and_evaluate(lrate=0.001, l1=0.0, l2=0.0, num_hidden=16):
    with tf.device("/device:CPU:0"):
        data_augmentation = Sequential(
            [
                layers.RandomCrop(
                    height=IMG_HEIGHT // 2,
                    width=IMG_WIDTH // 2,
                    input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
                ),
                layers.RandomFlip(mode="horizontal", name="random_lr_flip/none"),
                layers.RandomBrightness(factor=0.2, value_range=(0, 1)),
                layers.RandomContrast(factor=0.2),
            ]
        )

    train_dataset = (
        preprocessed_dataset(TRAIN_URL)
        .map(
            lambda image, label: (data_augmentation(image), label),
            num_parallel_calls=AUTOTUNE,
        )
        .batch(BATCH_SIZE)
        .prefetch(AUTOTUNE)
    )
    valid_dataset = (
        preprocessed_dataset(VALID_URL)
        .map(
            lambda image, label: (data_augmentation(image), label),
            num_parallel_calls=AUTOTUNE,
        )
        .batch(BATCH_SIZE)
        .prefetch(AUTOTUNE)
    )

    regularizer = regularizers.l1_l2(l1, l2)
    layers_ = [
        hub.KerasLayer(
            MODEL_URL,
            trainable=False,
            input_shape=(IMG_HEIGHT // 2, IMG_WIDTH // 2, IMG_CHANNELS),
            name="mobilenet_embedding",
        ),
        layers.Dense(
            num_hidden,
            kernel_regularizer=regularizer,
            kernel_initializer="he_normal",
            activation="elu",
            name="dense_hidden",
        ),
        layers.Dense(
            len(CLASS_NAMES),
            kernel_regularizer=regularizer,
            activation="softmax",
            name="flower_probs",
        ),
    ]

    model = Sequential(layers_, name="flower_classification")

    model.compile(
        optimizer=optimizers.Adam(learning_rate=lrate),
        loss=losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=["accuracy"],
    )

    print(model.summary())

    # Callbacks
    exponential_decay_fn = learning_rate_functions.exponential_decay_with_warmup(
        lr_start=lrate / 2,
        lr_max=lrate,
        lr_min=lrate / 10,
        lr_rampup_epochs=NUM_EPOCHS // 10,
        lr_sustain_epochs=NUM_EPOCHS // 10,
        lr_exp_decay=0.25,
    )
    lr_scheduler = callbacks.LearningRateScheduler(exponential_decay_fn)
    logdir = get_logdir()
    tensorboard_cb = callbacks.TensorBoard(log_dir=logdir)
    early_stop_cb = callbacks.EarlyStopping(patience=5, restore_best_weights=False)
    checkpoint_cb = callbacks.ModelCheckpoint(
        filepath="../../flowers_classifier/flower_classifier.h5", save_best_only=True
    )
    callbacks_ = [lr_scheduler, tensorboard_cb, early_stop_cb, checkpoint_cb]

    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=NUM_EPOCHS,
        callbacks=callbacks_,
    )

    return model


In [10]:
model = train_and_evaluate()

Model: "flower_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_embedding (KerasL  (None, 1280)             2257984   
 ayer)                                                           
                                                                 
 dense_hidden (Dense)        (None, 16)                20496     
                                                                 
 flower_probs (Dense)        (None, 5)                 85        
                                                                 
Total params: 2,278,565
Trainable params: 20,581
Non-trainable params: 2,257,984
_________________________________________________________________
None
Epoch 1/150
6/6 [==============================] - 190s 31s/step - loss: 2.1564 - accuracy: 0.2089 - val_loss: 1.7724 - val_accuracy: 0.3368 - lr: 5.0000e-04
Epoch 2/150
6/6 [==============================] - 216s 36s/step 

In [11]:
%load_ext tensorboard

In [13]:
%tensorboard --logdir=../../reports/logs


Reusing TensorBoard on port 6006 (pid 51664), started 7:31:42 ago. (Use '!kill 51664' to kill it.)